In [27]:
import pandas as pd
import re
import yaml

In [7]:
df=pd.read_csv('data_500.csv')
df

,text_cleaned,Loại BDS,Hướng nhà,Pháp lý,Nội thất,Vị trí so với biển,Tình trạng dân cư xung quanh,Hình dáng nhà,Khu vực để xe,Loại hẻm,...,Khả năng thoát nước,Vị trí,Vỉa hè,Giá,Diện tích,Kích thước,Số phòng ngủ,Số toilet,Số tầng,Đường trước nhà
0,chủ gởi bán mt tặng nhà cấp 4 đường tốt động p...,nhà,hướng đông bắc,NaN,NaN,gần biển,NaN,NaN,NaN,NaN,...,NaN,mặt tiền,có,"giá 2,7 tỷ",dt 70m,ngang 5m,NaN,NaN,NaN,NaN
1,hàng độc quyền chạy nhà chính chủ cần bán nhà ...,nhà,hướng đông nam,Chính chủ,NaN,NaN,đông đúc,NaN,NaN,NaN,...,NaN,NaN,có,giá 6.950 tỷ,diện tích 100m2,(5 20m),4 phòng ngủ,4wc,4.5 tầng,đường 10.5m
2,chính chủ gửi bán nhà 2 tầng 2 mê mặt tiền đườ...,nhà,hướng đông bắc,Chính chủ,NaN,NaN,đông đúc,NaN,NaN,NaN,...,NaN,mặt tiền,NaN,giá bán 3ty650,dt 90m2,(5 x 18),3pn,2wc,2 tầng,NaN
3,cần bán lô đất vườn xã hoà nimh .hoà vang .đà ...,đất,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,giá bán 6ty x x,dt 9900m2,NaN,NaN,NaN,NaN,NaN
4,chính chủ cần bán đất kiệt 87 hoàng văn thái. ...,đất,NaN,Chính chủ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,kiệt,NaN,giá 1ty600tr,diện tích 72m,( 5 x 14.4),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,"lô mới keng, 935m2 đất trồng cây hằng năm sát ...",đất,NaN,NaN,NaN,NaN,NaN,nở hậu,NaN,NaN,...,NaN,NaN,NaN,giá 670 triệu,NaN,"ngang 8,7m",NaN,NaN,NaN,NaN
396,bán lô đất xây nhà vườn đà nẵng 2 mặt tiền giá...,nhà,hướng đông bắc,Sổ hồng,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,mặt tiền,NaN,giá rẻ 1 tỷ 270 triệu,diện tích 413m2,ngang 24m,NaN,NaN,NaN,NaN
397,nhượng lại căn hộ fpt plaza 2 mặt tiền võ chí ...,căn hộ,NaN,Sổ hồng,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,mặt tiền,NaN,NaN,NaN,NaN,2 phòng ngủ,2 wc,NaN,NaN
398,view đẹp tận hưởng tự nhiên mở cửa sổ và thả m...,NaN,NaN,Sổ hồng,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
bien_patterns =yaml.safe_load(open("pattern/BIEN.yaml", 'r', encoding='utf8'))['rules']

In [20]:
def extract(text,patterns):
    for pattern in patterns:
        types = types = pattern['type']
        for _pattern in pattern['regex']:
            match = re.search(_pattern, str(text))
            if match:
                return types, match.group().strip()
    return None

In [21]:
def spec_extractor_multitype(text, patterns):
    value = None
    check_out = None
    lst_3words = []
    lst_5words = []
    lst_7words = []

    types, value = extract(text, patterns)

    if value == None:
        return None, None, None, None, None

    # 3 words BI entity
    pattern_3ws = r"\W*((?:\w+\W*){0,1})\W*" + re.escape(value) + r"\W*((?:\w+\W*){0,1})\W*"
    value_3ws = re.search(pattern_3ws, text, re.IGNORECASE).group()
    lst_3words.append(value_3ws)

    # 4 words BI entity
    pattern_5ws = "\W*((?:\w+\W*){0,5})\W*"+ value + "\W*((?:\w+\W*){0,5})\W*"
    value_5ws = re.search(pattern_5ws, text, re.IGNORECASE).group()
    lst_5words.append(value_5ws)

    # 7 words BI entity
    pattern_7ws = "\W*((?:\w+\W*){0,7})\W*"+ value + "\W*((?:\w+\W*){0,7})\W*"
    value_7ws = re.search(pattern_7ws, text, re.IGNORECASE).group()
    lst_7words.append(value_7ws)

    return text, types, list(set(lst_3words))[0], list(set(lst_5words))[0], list(set(lst_7words))[0]

In [22]:
texts = []
types = []
w3s = []
w5s = []
w7s = []

for text in df.text_cleaned.to_list():
    try:
        text, _type, w3, w5, w7 = spec_extractor_multitype(text_cleaned, bien_patterns)

        texts.append(text)
        types.append(_type)
        w3s.append(w3)
        w5s.append(w5)
        w7s.append(w7)
    except:
        pass

In [24]:
w5s

[]

In [25]:
data = pd.DataFrame({
    'text' : texts,
    'type' : types,
    '3 words' : w3s,
    '5 words' : w5s,
    '7 words' : w7s
})

In [26]:
data

,text,type,3 words,5 words,7 words
